# **Rag System using Llama2**

In [2]:
!pip install pypdf

In [3]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [4]:
#Embedding
!pip install sentence_transformers

In [5]:
%pip install llama-index-llms-huggingface
!pip install llama-index --upgrade


In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [7]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

[Document(id_='2cc9f114-12d0-4666-a14e-f17930254d1c', embedding=None, metadata={'page_label': '1', 'file_name': 'Abuse_Handling.pdf', 'file_path': '/content/data/Abuse_Handling.pdf', 'file_type': 'application/pdf', 'file_size': 53687, 'creation_date': '2025-09-01', 'last_modified_date': '2025-09-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# Abuse Handling \n \n## Section 3.1 – Abuse Categories \n- Phishing websites \n- Malware distribution \n- Spam and botnet control \n \n## Section 3.2 – Escalation \n- All abuse-related tickets must be escalated to the Abuse Team. \n- Escalation email: abuse@company.example \n \n## Section 3.3 –

In [8]:
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
Answer the question in the following format:
  "answer",
  "references",
  "action_required":
"""

#Deafult Format Supported by LLAMA2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")


Here we connect the HuggingFace CLI(command line interface) with our pipeline. A token is generated using a huggingface account and is added as a git credential.

In [9]:
!!huggingface-cli login

["\x1b⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.\x1b",
 '',
 '    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|',
 '    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|',
 '    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|',
 '    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|',
 '    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|',
 '',
 '    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.',
 '    Setting a new token will erase the existing one.',
 '    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .',


For the following to work, make sure that proper permissions are requested from Meta on the HuggingFace website: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

Alternatively, if permissions are not granted use:
"mistralai/Mistral-7B-Instruct-v0.3". It is free and requires the user to agree to its terms and user rights are granted immediately.

In [10]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True} #We load it in 8bit since quantization takes place
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
!pip install -U langchain-community

In [12]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

ImportError: cannot import name 'ServiceContext' from 'llama_index' (unknown location)